In [4]:
from random import randint

In [5]:
from mip import *

Using Python-MIP package version 1.5.0


In [27]:
""" 
Parameters
------------
n : int 
    the number of triples to be generate
l : int
    lower bound of the values in the triples
u : int
    upper bound of the values in the triples
    
Returns
-------
pop : set((int,int,int)) 
      a set of triples
"""
def generate_pop(n,l,u):
    pop = set()
    while(len(pop) < n):
        pop.add((randint(l,u),randint(l,u),randint(l,u)))
    return pop

"""
Parameters
----------
t1 : (int, int, int )
    the first triple
t2 : (int, int, int )
    the second triple
    
Returns
-------
true if t1 WeaklyPD t2 and false otherwise 
"""
def PD_dominates(t1,t2):
    return (t1[0]>=t2[0] and t1[1]>=t2[1] and t1[2]>=t2[2])
                
                
"""
Parameters
----------
pop : set((int,int,int))
      a set of triples

Returns
-------
X : set((int,int,int))
"""
def generate_X_PD(pop):
    X = set()
    
    #X starts with a random subset of pop
    for t in pop:                                                                                                                    
        if randint(0, 1):
            X.add(t)
    
    #We complement X to ensure PD closure
    Xcp = set(X)
    for t1 in Xcp:
        for t2 in (pop - Xcp):
            if PD_dominates(t2,t1):
                X.add(t2)
    return X

"""
Parameters
----------
pop :
X :

Returns 
-------
"""
def test_LP(pop,X):
    t_list = list(set([t for (_,_,t) in pop]))
    Tau = dict()
    
    m = Model(sense=MINIMIZE, solver_name=CBC)
    m.verbose = 0
    for t in t_list:
        Tau[t]=m.add_var(var_type=CONTINUOUS, lb=0)
    
    for i in range(len(t_list)):
        for j in range(i+1,len(t_list)):
            if t_list[i] < t_list[j]:
                m += Tau[t_list[i]] <= Tau[t_list[j]]
            else:
                m += Tau[t_list[j]] <= Tau[t_list[i]]
    
    for t1 in X:
        for t2 in (pop-X):
            if t2[0] >= t1[0]:
                m += t1[1] + Tau[t1[2]] >= t2[1] + Tau[t2[2]]
    m.objective = minimize(xsum(Tau[t] for t in t_list))
    status = m.optimize()
    return (status == OptimizationStatus.INFEASIBLE)

In [38]:
n = 5
ub= 50
pop = generate_pop(n,0,ub)
X = generate_X_PD(pop)
while(not test_LP(pop,X)):
    pop = generate_pop(n,0,ub)
    X = generate_X_PD(pop)
    
print("pop : " + str(pop))
print("X : " + str(X))

pop : {(36, 18, 42), (38, 29, 4), (12, 34, 3), (34, 28, 46), (47, 30, 49)}
X : {(47, 30, 49), (12, 34, 3), (36, 18, 42)}
